### 数据类型 #https://www.postgresql.org/docs/9.2/datatype.html


In [ ]:
###创建关系数据表与 "用户,音乐" 字典数据表


drop table if exists tbl_user;
drop table if exists tbl_music;
drop table if exists tbl_lmf_data;

create table tbl_user(id bigserial,userid varchar(10));
create table tbl_music(id bigserial,musicid varchar(10));
create table tbl_lmf_data(row integer,col integer,val float8);

##插入数据
insert into tbl_user(userid) values ('u0'),('u1'),('u2'),('u3'),('u4'),('u5'),('u6'),('u7'),('u8'),('u9'),('u10');
insert into tbl_music(musicid) values ('m1'),('m2'),('m3'),('m4'),('m5'),('m6'),('m7'),
('m8'),('m9'),('m10'),('m11'),('m12'),('m13'),('m14'),('m15');
#插入评论关系
insert into tbl_lmf_data values (1,1,5),(1,6,-5),(1,9,5),(1,11,3),(1,12,1),(1,13,5);
insert into tbl_lmf_data values (2,4,3),(2,9,3),(2,13,4);
insert into tbl_lmf_data values (3,3,1),(3,4,2),(3,6,-5),(3,7,4),(3,11,-2),(3,12,-2),(3,13,-2);
insert into tbl_lmf_data values (4,2,4),(4,3,4),(4,4,3),(4,7,-2),(4,9,-5),(4,12,3);
insert into tbl_lmf_data values (6,2,5),(6,3,-5),(6,5,-5),(6,7,4),(6,8,3),(6,11,4);
insert into tbl_lmf_data values (7,3,4),(7,6,3),(7,9,4);
insert into tbl_lmf_data values (8,2,-2),(8,6,5),(8,11,4),(8,12,4),(8,13,-2);
insert into tbl_lmf_data values (9,2,-2),(9,6,5),(9,8,5),(9,11,4),(9,13,-2);

### 矩阵分解https://madlib.apache.org/docs/latest/group__grp__lmf.html

In [ ]:
select madlib.lmf_igd_run( 
             'tbl_lmf_model',
             'tbl_lmf_data',
             'row',
             'col',
             'val',
             11,
             16,
             7,
             0.1,
             1,
             10,
             1e-9
           );

### 矩阵转表 https://madlib.apache.org/docs/latest/group__grp__vec2cols.html

In [ ]:
create table tbl_U as 
select d1,d2,matrix_u[d1][d2] val from 
    (select matrix_u,generate_series(1,array_upper(matrix_u,1)) d1 from tbl_lmf_model) t1 ,
    (select generate_series(1,array_upper(matrix_u,2)) d2 from tbl_lmf_model) t2;
    
create table tbl_V as 
select d1,d2,matrix_v[d1][d2] val from 
    (select matrix_v,generate_series(1,array_upper(matrix_v,1)) d1 from tbl_lmf_model) t1 ,
    (select generate_series(1,array_upper(matrix_v,2)) d2 from tbl_lmf_model) t2;

### 矩阵乘法 https://madlib.apache.org/docs/latest/group__grp__matrix.html#operations

In [ ]:
select madlib.matrix_mult('tbl_U','row=d1,col=d2,val=val',
                          'tbl_V','row=d1,col=d2,val=val,trans=true',
                          'tbl_result','row=row,col=col');
#在转为稀疏矩阵
select madlib.matrix_sparsify('tbl_result','row=row,val=val','tbl_sparsify_result','row=rowid,col=colid,val=val');

In [ ]:
##查询出 人对电影 的喜欢程度

select u.userid,m.musicid,a.rn,a.val from
(select rowid,colid,row_number() over (partition by rowid order by val desc) rn,val 
from tbl_sparsify_result t1 
 where not exists (select 1 from tbl_lmf_data t2 where t2.row=t1.rowid and t2.col=t1.colid)) a,
tbl_user u,
tbl_music m
where u.uid=a.rowid and m.id=a.colid
order by u.userid,a.val desc;